In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from utils import delete_nones, encode_ids

## Load and Preprocess data

In [4]:
frame = pd.read_csv("./data/clear_data.scv")

In [5]:
frame = delete_nones(frame)
frame = encode_ids(frame)

frame shape =  (1117300, 12)
clear frame shape =  (995791, 12)


In [ ]:
# TODO: move into main
print('frame shape = ', frame.shape)
print('num unique drivers = ', np.array(frame.driver_id.unique()).shape[0])
print('num unique users = ', np.array(frame.user_id.unique()).shape[0])
print('num unique rides = ', np.array(frame.ride_id.unique()).shape[0])

## Compute Features

In [1]:
import xgboost as xgb
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.preprocessing import normalize
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split

In [ ]:
# read in data
dtrain = xgb.DMatrix(np.array(X_train_add[[]]), 
                     label=np.array(y_train))

dtest = xgb.DMatrix(np.array(X_test_add[[]]),
                    label=np.array(y_test))

params_scope = [{'booster': 'dart',
                 'max_depth': 5, 
                 'learning_rate': 0.1,
                 'objective': 'multi:softmax', 
                 'silent': True,
                 'normalize_type': 'tree',
                 'rate_drop': 0.1,
                 'skip_drop': 0.5,
                 'num_class': 3},

                 {'booster': 'dart',
                 'max_depth': 3, 
                 'learning_rate': 0.01,
                 'objective': 'multi:softmax', 
                 'silent': True,
                 'normalize_type': 'tree',
                 'rate_drop': 0.1,
                 'skip_drop': 0.8,
                 'num_class': 3},

                 {'booster': 'gblinear',
                 'max_depth': 8, 
                 'learning_rate': 0.001,
                 'objective': 'multi:softmax', 
                 'silent': True,
                 'normalize_type': 'forest',
                 'num_class': 3},

                {'booster': 'dart',
                 'max_depth': 4, 
                 'learning_rate': 0.1,
                 'objective': 'multi:softmax', 
                 'silent': True,
                 'normalize_type': 'tree',
                 'rate_drop': 0.2,
                 'skip_drop': 0.6,
                 'num_class': 3}
                ]

In [ ]:
scores = []
scores_cv = []

for param_num, params in enumerate(params_scope):
    print()
    print('##############################')
    print('###           {}           ###'.format(param_num))
    print('##############################')
    bst = xgb.train(param, dtrain)
    
    y_pred = bst.predict(dtest)
    score = metrics.accuracy_score(y_test, y_pred)
    scores.append(score)
    print('Score = ', score)
    
    
    score_cv = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
    scores_cv.append(score_cv)
    
    print('score_cv = ', scores_cv)

In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.preprocessing import normalize
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split

from utils import delete_nones, encode_ids
from features import time_discretization, find_hexagon_id_of_pickup_coords, create_city_hexagons

/home/mak7/PycharmProjects/MURA/mur/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# TODO: insert data cleaning
frame = pd.read_csv("./data/clear_data.scv")

In [7]:
# DATA PREPROCESSING
frame = delete_nones(frame)
frame = encode_ids(frame)
# delete canceled requests
frame = frame[(frame['canceled_by_client'] == 0) & (frame['canceled_by_driver'] == 0)]

frame shape =  (995791, 12)
clear frame shape =  (995791, 12)


In [8]:
# get hexagon frame
# TODO: pass params of city
hexagon_frame = create_city_hexagons()

In [9]:
hexagon_frame.head()

,hexagon_id,latitude1,latitude2,latitude3,latitude4,latitude5,latitude6,longitude1,longitude2,longitude3,longitude4,longitude5,longitude6
0,1,49.843030,49.838817,49.835233,49.835861,49.840073,49.843658,24.025218,24.022414,24.027112,24.034614,24.037419,24.032721
1,2,49.835233,49.831020,49.827436,49.828064,49.832276,49.835861,24.027112,24.024308,24.029005,24.036506,24.039311,24.034614
2,3,49.838189,49.833976,49.830392,49.831020,49.835233,49.838817,24.014911,24.012108,24.016806,24.024308,24.027112,24.022414
3,4,49.845985,49.841772,49.838189,49.838817,49.843030,49.846614,24.013016,24.010212,24.014911,24.022414,24.025218,24.020519
4,5,49.847870,49.843658,49.840073,49.840700,49.844912,49.848497,24.035526,24.032721,24.037419,24.044921,24.047727,24.043029


In [10]:
# TODO: add city column
# FEATURE GENERATION
time_period_frame = time_discretization(frame=frame)
frame['week_day'], frame['weekend'], frame['month'], frame['year'] = time_period_frame['week_day'], \
                                                                 time_period_frame['weekend'], \
                                                                 time_period_frame['month'], \
                                                                 time_period_frame['year']

In [11]:
frame = frame.reset_index(drop=True)

In [9]:
frame.head()

,ride_id,created_at,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,user_id,driver_id,ride_distance,canceled_by_client,canceled_by_driver,ride_to_suburb,week_day,weekend,month,year
0,0,2018-12-14 23:00:00.043000000,49.824671,24.013246,49.784729,24.060303,45133,2726,7.97,0,0,0,4.0,False,12.0,2018.0
1,1,2018-08-25 21:40:52.537000000,49.839718,23.994425,49.840610,24.029135,122255,1942,3.17,0,0,0,4.0,False,1.0,2019.0
2,2,2019-01-18 09:18:44.377000000,49.837013,24.034357,49.832710,23.999216,20280,665,3.39,0,0,0,4.0,False,1.0,2019.0
3,3,2018-10-03 16:53:48.097000000,49.871124,23.957130,49.858883,24.019121,77669,1456,6.74,0,0,0,6.0,True,9.0,2018.0
4,4,2019-01-25 06:25:31.730000000,49.785679,24.055679,49.842602,24.000990,57120,3846,13.16,0,0,0,3.0,False,12.0,2018.0


In [31]:
from matplotlib import path
from tqdm import tqdm

In [47]:
def find_hexagon_id_of_pickup_coords(hexagon_frame, frame):
    # TODO: decide what to do with pickoff_lat and pickoff_lng
    pickup_coords = np.array(frame[['pickup_lat', 'pickup_lng']])

    hexagon_arr = np.array(hexagon_frame)[:, 1:]
    hexagon_ids = np.array(hexagon_frame['hexagon_id'])

    hexagon_pickup_ids = []

    for coord_ind, pickup_coord in tqdm(enumerate(pickup_coords)):
        # TODO: use kNN for optimization
        for hexagon_ind, hexagon in enumerate(hexagon_arr):
            hexagon = np.array([hexagon[0:6], hexagon[6:]]).reshape(-1, 6).T
            hexagon_path = path.Path(hexagon)

            if hexagon_path.contains_points([(pickup_coord[0], pickup_coord[1])]):
                hexagon_pickup_ids.append(hexagon_ids[hexagon_ind])
                break
            elif hexagon_ind == hexagon_arr.shape[0]-1:
                hexagon_pickup_ids.append(None)
            else:
                continue

    return hexagon_pickup_ids

In [29]:
small_frame = frame[0:5000]
print(small_frame.shape)
small_frame.head()

(5000, 16)


,ride_id,created_at,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,user_id,driver_id,ride_distance,canceled_by_client,canceled_by_driver,ride_to_suburb,week_day,weekend,month,year
0,0,2018-12-14 23:00:00.043000000,49.824671,24.013246,49.784729,24.060303,45133,2726,7.97,0,0,0,4.0,False,12.0,2018.0
1,1,2018-08-25 21:40:52.537000000,49.839718,23.994425,49.840610,24.029135,122255,1942,3.17,0,0,0,4.0,False,1.0,2019.0
2,2,2019-01-18 09:18:44.377000000,49.837013,24.034357,49.832710,23.999216,20280,665,3.39,0,0,0,4.0,False,1.0,2019.0
3,3,2018-10-03 16:53:48.097000000,49.871124,23.957130,49.858883,24.019121,77669,1456,6.74,0,0,0,6.0,True,9.0,2018.0
4,4,2019-01-25 06:25:31.730000000,49.785679,24.055679,49.842602,24.000990,57120,3846,13.16,0,0,0,3.0,False,12.0,2018.0


In [43]:
small_frame['hexagon_id'] = find_hexagon_id_of_pickup_coords(hexagon_frame, small_frame)

4438it [00:08, 532.07it/s]


In [44]:
small_frame.head()

,ride_id,created_at,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,user_id,driver_id,ride_distance,canceled_by_client,canceled_by_driver,ride_to_suburb,week_day,weekend,month,year,hexagon_id
0,0,2018-12-14 23:00:00.043000000,49.824671,24.013246,49.784729,24.060303,45133,2726,7.97,0,0,0,4.0,False,12.0,2018.0,34
1,1,2018-08-25 21:40:52.537000000,49.839718,23.994425,49.840610,24.029135,122255,1942,3.17,0,0,0,4.0,False,1.0,2019.0,35
2,2,2019-01-18 09:18:44.377000000,49.837013,24.034357,49.832710,23.999216,20280,665,3.39,0,0,0,4.0,False,1.0,2019.0,1
3,5,2018-09-30 04:15:57.223000000,49.798859,24.061859,49.813759,23.959866,95136,3114,9.30,0,0,0,4.0,False,1.0,2019.0,105
4,6,2018-12-20 02:28:31.017000000,49.844044,24.024796,49.843155,23.972368,12655,4259,4.55,0,0,0,1.0,False,2.0,2019.0,7


In [47]:
small_frame = small_frame.dropna()
small_frame = small_frame.reset_index(drop=True)

features = small_frame.groupby(['week_day', 'month', 'year', 'hexagon_id']).count()
feature_frame = features.reset_index()
feature_frame = feature_frame[['week_day', 'month', 'year', 'hexagon_id', 'ride_id']]
feature_frame.columns = ['week_day', 'month', 'year', 'hexagon_id', 'num_in']
feature_frame['weekend'] = [True if weekday in (5, 6) else False for weekday in feature_frame.week_day.values]
feature_frame['ride_distance'] = small_frame.groupby(['week_day', 'month', 'year', 'hexagon_id']).sum()['ride_distance'].values


In [48]:
feature_frame.head()

,week_day,month,year,hexagon_id,num_in,weekend,ride_distance
0,0.0,1.0,2019.0,1,15,False,79.589999
1,0.0,1.0,2019.0,2,3,False,13.830000
2,0.0,1.0,2019.0,4,3,False,11.550000
3,0.0,1.0,2019.0,5,1,False,12.330000
4,0.0,1.0,2019.0,6,2,False,5.690000


In [ ]:
def compute_load_coef():
    

In [49]:
feature_frame['load_coef'] = feature_frame.num_in / feature_frame.num_in.max()

In [50]:
feature_frame.head()

,week_day,month,year,hexagon_id,num_in,weekend,ride_distance,load_coef
0,0.0,1.0,2019.0,1,15,False,79.589999,0.625000
1,0.0,1.0,2019.0,2,3,False,13.830000,0.125000
2,0.0,1.0,2019.0,4,3,False,11.550000,0.125000
3,0.0,1.0,2019.0,5,1,False,12.330000,0.041667
4,0.0,1.0,2019.0,6,2,False,5.690000,0.083333


In [51]:
feature_frame.load_coef.max()

1.0

In [119]:
feature_frame.shape

(3394, 6)

In [123]:
feature_frame.head()

week_day  month  year    hexagon_id
0.0       1.0    2019.0  1.0           244.559999
                         2.0             3.110000
                         3.0            30.570001
                         4.0            45.670000
                         5.0            40.340000
Name: ride_distance, dtype: float64